# (Kernel) Ridge Regression
Download the Spotify Tracks Dataset and perform ridge regression to predict the tracks’ popularity. Note that this dataset contains both numerical and categorical features. The student is thus required to follow these guidelines:
- first, train the model using only the numerical features,
- second, appropriately handle the categorical features (for example, with one-hot encoding or other techniques) and use them together with the numerical ones to train the model, in both cases, experiment with different training parameters, 
- use 5-fold cross validation to compute your risk estimates, thoroughly discuss and compare the performance of the model

The student is required to implement from scratch (without using libraries, such as Scikit-learn) the code for the ridge regression, while it is not mandatory to do so for the implementation of the 5-fold cross-validation.

Optional: Instead of regular ridge regression, implement kernel ridge regression using a Gaussian kernel.


## TODOS
 - for each alpha, save the computed loss in order to draw a graph 
 - try to avoid some features for seeing whether or not the loss decreases
 - compare our cv with the skitlearn cv
 - implementing a function that finds the predictor with the lowest loss in an alpha list

# Initialization


In [ ]:
if "google.colab" in str(get_ipython()):
    !git clone https://github.com/lukebella/SpotifyRegression.git
    !mv SpotifyRegression/* .
    !rm -fr SpotifyRegression

In [ ]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d maharshipandya/-spotify-tracks-dataset
!unzip -n ./data/-spotify-tracks-dataset.zip -d ./data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [ ]:
# Open the dataset

train_set = "data/dataset.csv"

dataset_df = pd.read_csv(train_set).drop(columns='Unnamed: 0')
dataset_df

# Split the dataset

In [ ]:
# Split the dataset into training set and test set

np.random.seed(0)
mask = np.random.rand(len(dataset_df))<0.7

train_df = dataset_df[mask]
test_set = dataset_df[~mask]

# y_train_df = train_df[["popularity"]]
# y_train_df

In [ ]:
loudness_norm = dataset_df["loudness"]
tempo_norm = dataset_df["tempo"]

tempo_norm = tempo_norm/tempo_norm.max(axis=0)

#(abs(min(col)) + i)/max+abs(min(col))

min_loud = abs(loudness_norm.min(axis=0))
max_loud = abs(loudness_norm.max(axis=0))

loudness_norm = (min_loud+loudness_norm)/(min_loud+max_loud)
dataset_df["loudness"] = loudness_norm
dataset_df["tempo"] = tempo_norm

#print(dataset_df["loudness"])

In [ ]:
# Numerical features
numerical_df = dataset_df[["popularity", "duration_ms", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

train_num_df = numerical_df[mask]
test_num_df = numerical_df[~mask]


In [ ]:
# Categorical features
# One-hot encoding the categorical features using get_dummies
categorical_df = pd.get_dummies(dataset_df.drop(columns= ["track_id", "artists", "album_name", "track_name"]), 
                                columns = ['explicit','key', 'mode', 'time_signature', 'track_genre'], dtype=int)

train_cat_df = categorical_df[mask]
test_cat_df = categorical_df[~mask]


# Defining functions

In [ ]:
# Create the hyperplane using regular ridge regression
def ridge_regression(alpha, train_set):
    y = train_set[["popularity"]]
    train_set = train_set.drop(columns='popularity')
    n_rows, n_cols = train_set.shape  # Get the dimensions of the input matrix s
    s_t = train_set.transpose()  # Transpose of matrix s
    
    # Calculate the identity matrix with the appropriate size
    identity = np.identity(n_cols)
    
    # Calculate the ridge regression coefficients using matrix operations
    w = (np.linalg.inv(alpha * identity + np.dot(s_t, train_set)).dot(s_t)).dot(y) 
    
    # Convert the coefficients to a DataFrame for better presentation
    w_df = pd.DataFrame(w, columns=["Values"], index=train_set.columns)
    
    return w_df

In [ ]:
# Predict the popularity of a track x using an hyperplane w
def predict(w, x):
    pred = w.transpose().dot(x.drop(labels='popularity'))[0]
    #pred = max(0, pred)
    #return min(100, pred)
    return pred
    

In [ ]:
# Compute the average square loss of the hyperplane w
def avg_square_loss(w, test_set):
    y = test_set[["popularity"]]
    test_set = test_set.drop(columns='popularity')
    # Convert the DataFrame to a numpy array
    x = test_set.values  
    # Calculate predictions for all rows at once
    predictions = np.dot(x, w)
    
    squared_diff = (predictions -  y)**2
    total_loss = np.sum(squared_diff)
    return total_loss.values[0]/test_set.shape[0]

# Ridge Regression using only numerical features

In [ ]:
# Compute the hyperplane for the numercal dataset
result_numeric = ridge_regression(0.5, train_num_df)
result_numeric

In [ ]:
# Predict the first row of the training set
predicted_y = predict(result_numeric, train_num_df.iloc[4])
print(f"Predicted y: \t{predicted_y}\nReal y: \t{train_num_df.iloc[4]['popularity']}")

In [ ]:
# Compute the Average square loss of the hyperplane (numerical)
print("Average square loss: ", avg_square_loss(result_numeric, test_num_df))

In [ ]:
alphas = 10**np.linspace(10, -2, 100)*0.5

num_train_losses = []
num_test_losses = []

for a in alphas:
    ridge = ridge_regression(a, train_num_df)
    num_train_losses.append(avg_square_loss(ridge, train_num_df))
    num_test_losses.append(avg_square_loss(ridge, test_num_df))

In [ ]:
plt.title('MSE numerical features')
plt.plot(alphas, num_train_losses, label='Training accuracy')
plt.plot(alphas, num_test_losses, label='Testing accuracy')
plt.xscale('log')
plt.legend()
plt.xlabel('Alpha')
plt.ylabel('Mean squared error')
plt.show()

# Scikitlearn Ridge regression on numerical features


In [ ]:
#Scikitlearn Ridge regression

alphas = 10**np.linspace(10, -2, 100)*0.5

sk_num_train_losses = []
sk_num_test_losses = []

for a in alphas:
    clf = Ridge(alpha = a)
    clf.fit(train_num_df.drop(columns='popularity'), train_num_df['popularity'])
    sk_num_train_losses.append(mean_squared_error(train_num_df['popularity'], clf.predict(train_num_df.drop(columns='popularity'))))
    sk_num_test_losses.append(mean_squared_error(test_num_df['popularity'], clf.predict(test_num_df.drop(columns='popularity'))))




In [ ]:
plt.title('ScikitLearn MSE numerical features')
plt.plot(alphas, sk_num_train_losses, label='Training accuracy')
plt.plot(alphas, sk_num_test_losses, label='Testing accuracy')
plt.xscale('log')
plt.legend()
plt.xlabel('Alpha')
plt.ylabel('Mean squared error')
plt.show()

# Ridge regression considering all features

In [ ]:
# Compute the hyperplane for the numercal dataset
result_categoric = ridge_regression(0.5, train_cat_df)
result_categoric

In [ ]:
# Predict the first row of the training set
predicted_y = predict(result_categoric, train_cat_df.iloc[0])
print(f"Predicted y: \t{predicted_y}\nReal y: \t{train_cat_df.iloc[0]['popularity']}")

In [ ]:
# Compute the Average square loss of the hyperplane (categorical)
print("Average square loss: ", avg_square_loss(result_categoric, test_cat_df))


In [ ]:
alphas = 10**np.linspace(10, -2, 100)*0.5

cat_train_losses = []
cat_test_losses = []

for a in alphas:
    ridge = ridge_regression(a, train_cat_df)
    cat_train_losses.append(avg_square_loss(ridge, train_cat_df))
    cat_test_losses.append(avg_square_loss(ridge, test_cat_df))

In [ ]:
plt.title('MSE on all features')
plt.plot(alphas, cat_train_losses, label='Training accuracy')
plt.plot(alphas, cat_test_losses, label='Testing accuracy')
# plt.plot(alphas, sk_cat_train_losses, label='SK Training accuracy')
# plt.plot(alphas, sk_cat_test_losses, label='SK Testing accuracy')
plt.xscale('log')
plt.legend()
plt.xlabel('Alpha')
plt.ylabel('Mean squared error')
plt.show()

# ScikitLearn Ridge regression on all features

In [ ]:
#Scikitlearn Ridge regression

alphas = 10**np.linspace(10, -2, 100)*0.5

sk_cat_train_losses = []
sk_cat_test_losses = []

for a in alphas:
    clf = Ridge(alpha = a)
    clf.fit(train_cat_df.drop(columns='popularity'), train_cat_df['popularity'])
    sk_cat_train_losses.append(mean_squared_error(train_cat_df['popularity'], clf.predict(train_cat_df.drop(columns='popularity'))))
    sk_cat_test_losses.append(mean_squared_error(test_cat_df['popularity'], clf.predict(test_cat_df.drop(columns='popularity'))))


In [ ]:
plt.title('ScikitLearn MSE all features')
plt.plot(alphas, sk_cat_train_losses, label='Training accuracy')
plt.plot(alphas, sk_cat_test_losses, label='Testing accuracy')
plt.xscale('log')
plt.legend()
plt.xlabel('Alpha')
plt.ylabel('Mean squared error')
plt.show()

# Numerical vs All features

In [ ]:
plt.title('MSE: numerical features vs. all features')
plt.plot(alphas, num_train_losses, label='Num training accuracy')
plt.plot(alphas, num_test_losses, label='Num testing accuracy')
plt.plot(alphas, cat_train_losses, label='All training accuracy')
plt.plot(alphas, cat_test_losses, label='All testing accuracy')
plt.xscale('log')
plt.legend()
plt.xlabel('Alpha')
plt.ylabel('Mean squared error')
plt.show()

# (Nested) Cross Validation

In [ ]:
def cross_validation(k, dataset, alphas):

    # Return a df from an arraty of df excepr the i-th
    def get_set_except_i(dataset_array, i):
        return pd.concat(dataset_array[j] for j in range(len(dataset_array)) if i!=j)
    
    # Split the dataset into k parts
    dataset_array = np.array_split(dataset, k)
    
    losses = []

    for i in range(k):
        # In the i-th iteration, Si is the test and S\Si is the training
        test_cv = dataset_array[i] 
        train_cv = get_set_except_i(dataset_array, i)

        # Split the training set into a new training set and a valid set (nested CV)
        train_cv_array = np.array_split(train_cv, k-1)
        dev_cv = train_cv_array[0]
        nested_cv = get_set_except_i(train_cv_array, 0)
        
        # Find the best hyperparameter of your alphas
        loss = float("inf")
        alpha = 0
        for a in alphas:
            predictor = ridge_regression(a, nested_cv)

            local_loss = avg_square_loss(predictor, dev_cv)
            if loss > local_loss:
                loss = local_loss
                alpha = a
                
        # Compute k predictors and their losses
        prediction = ridge_regression(alpha, train_cv)
        losses.append(avg_square_loss(prediction, test_cv))

    #Find the avg loss of the predictors
    return np.mean(losses), prediction, alpha

In [ ]:
K = 5
alphas = 10**np.linspace(10, -2, 100)*0.5
loss_cv, pred_cv, a_cv = cross_validation(K, categorical_df[:1000], alphas)
print("Average loss with nested CV: ", loss_cv)
print("Best alpha with nested CV: ", a_cv)

# Average loss with nested CV:  499.37945948919185
# Best alpha with nested CV:  201.85086292982749

# ScikitLearn Ridge CV

In [ ]:
alphas = 10**np.linspace(10, -2, 100)*0.5

clf = RidgeCV(alphas = alphas, cv= 5)
clf.fit(categorical_df.drop(columns='popularity'), categorical_df['popularity'])

mean_squared_error(categorical_df['popularity'], clf.predict(categorical_df.drop(columns='popularity')))

#392.4801619277324


# Kernel Ridge Regression


In [ ]:
def gaussian_kernel(gamma, v1, v2):
    norm = (np.linalg.norm(v1 - v2))**2
    return [np.exp((norm)/-(2 * (gamma))), norm]   #(1/gamma*np.sqrt(2*np.pi)) *


def kernel_ridge_regression(dataset, alpha, gamma = 0):
    y = dataset["popularity"]
    dataset_values = dataset.drop(columns='popularity').values
    n_samples = dataset.shape[0]
    #print((np.linalg.norm(dataset_values[0]-dataset_values[1])**2)/2)
    kernel = np.zeros((n_samples, n_samples))
    if gamma == 0:
        gamma = (np.linalg.norm(dataset_values[0]-dataset_values[1])**2)/2 + 1
        for i in range(n_samples):
            for j in range(i,n_samples):
                cell, gamma_new = gaussian_kernel(gamma, dataset_values[i], dataset_values[j])
                kernel[i, j] = cell
                gamma = (gamma + gamma_new)/2
    else:
        for i in range(n_samples):
            for j in range(i,n_samples):
                cell, gamma_new = gaussian_kernel(gamma, dataset_values[i], dataset_values[j])
                kernel[i, j] = cell
    
    #we consider half of the datapoints since it is the 'specular'
    kernel = np.triu(kernel, 1) + kernel.transpose()  
    #print(kernel)
    identity = np.identity(n_samples)
    #print(y.transpose())
    #print(np.linalg.inv((alpha * identity + kernel)))
    w = y.transpose() @ np.linalg.inv((alpha * identity + kernel))
    #print(w)
    w_df = pd.DataFrame(w, columns=['weights'])
    return w_df


def kernel_predict(w, dataset, x, gamma = 0):
    x_values = x.drop(labels='popularity').values
    dataset_values = dataset.drop(columns='popularity').values
    #n_samples = dataset_values.shape[0]
    kernel_values = []

    if gamma == 0:
        gamma = (np.linalg.norm(dataset_values[0]-x_values)**2)/2 +1

        for x_i in dataset_values:
            kernel_value, new_gamma = gaussian_kernel(gamma, x_values, x_i)
            gamma = (new_gamma + gamma)/2
            kernel_values.append(kernel_value)
    else:
        for x_i in dataset_values:
            kernel_value, new_gamma = gaussian_kernel(gamma, x_values, x_i)
            kernel_values.append(kernel_value)
            
    kernel_values = np.array(kernel_values)
    prediction = w['weights'].dot(kernel_values)
    prediction = max(0, prediction)  #ReLU
    prediction = min(100, prediction)
    return prediction

def kernel_avg_square_loss(w, train_set, test_set, gamma=0):
    y = test_set[["popularity"]]
    predictions = test_set.apply(lambda r: kernel_predict(w,train_set,r, gamma), 1) 
    squared_diff = (predictions - y.transpose())**2
    return np.mean(squared_diff, axis=1)[0]



In [ ]:
train_set = train_cat_df[:1000]
x = categorical_df.iloc[90]
print(x['popularity'])
w = kernel_ridge_regression(train_set, 1)
print(kernel_predict(w, train_set, x))

loss = kernel_avg_square_loss(w, train_set, test_cat_df[:5000])
print("AVG Square loss: ", loss)
print("AVG loss: ", loss**(1/2))
# The predicted values must been adjusted, try to normalize all the values between 0 and 1 and multiply by 100 in the end for the popularity result

In [ ]:
train_set = train_cat_df[:1000]
x = categorical_df.iloc[90]
gamma = 10000000
print(x['popularity'])
w = kernel_ridge_regression(train_set, 1, gamma)
print(kernel_predict(w, train_set, x, gamma))

loss = kernel_avg_square_loss(w, train_set, test_cat_df[:1000], gamma)
print("AVG Square loss: ", loss)
print("AVG loss: ", loss**(1/2))
# The predicted values must been adjusted, try to normalize all the values between 0 and 1 and multiply by 100 in the end for the popularity result